In [0]:

# Loading table into a Spark DataFrame
df = spark.table("workspace.default.customer_transaction_analysis")

# Show a few rows
df.show(5)

# Print schema
df.printSchema()

# Total records and columns
print("Rows:", df.count())
print("Columns:", len(df.columns))

+--------------+-----------+----------------+-----------------+-------+--------+-------+------------+----------------+---------+
|transaction_id|customer_id|   txn_timestamp|merchant_category| amount|currency|channel|card_present|is_international|branch_id|
+--------------+-----------+----------------+-----------------+-------+--------+-------+------------+----------------+---------+
|       T077878|     C03736|05-10-2025 17:27|           Travel|2545.24|     INR| Online|           0|               0|     B008|
|       T067511|     C02842|09-06-2025 02:13|           Travel| 695.85|     INR| Online|           0|               0|     B036|
|       T095788|     C03066|27-10-2025 04:21|        Groceries|4226.46|     INR|    POS|           1|               0|     B039|
|       T059901|     C02970|29-06-2025 14:13|        Utilities| 595.82|     INR|    POS|           1|               0|     B032|
|       T099994|     C00096|13-06-2025 07:23|       E-commerce|  653.0|     INR|    POS|         

In [0]:
from pyspark.sql.functions import col, sum

# Count nulls for each column
df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()


+--------------+-----------+-------------+-----------------+------+--------+-------+------------+----------------+---------+
|transaction_id|customer_id|txn_timestamp|merchant_category|amount|currency|channel|card_present|is_international|branch_id|
+--------------+-----------+-------------+-----------------+------+--------+-------+------------+----------------+---------+
|             0|          0|            0|                0|     0|       0|      0|           0|               0|        0|
+--------------+-----------+-------------+-----------------+------+--------+-------+------------+----------------+---------+



In [0]:
df.describe(["amount"]).show()


+-------+------------------+
|summary|            amount|
+-------+------------------+
|  count|            100000|
|   mean|2611.4733810999815|
| stddev| 2509.808779598188|
|    min|            100.01|
|    max|          28733.07|
+-------+------------------+



In [0]:
df.selectExpr(
    "count(distinct transaction_id) as unique_txns",
    "count(distinct customer_id) as unique_customers",
    "count(distinct merchant_category) as merchant_categories",
    "count(distinct channel) as channels",
    "count(distinct branch_id) as branches"
).show()


+-----------+----------------+-------------------+--------+--------+
|unique_txns|unique_customers|merchant_categories|channels|branches|
+-----------+----------------+-------------------+--------+--------+
|     100000|            5000|                  9|       3|      50|
+-----------+----------------+-------------------+--------+--------+



In [0]:
df.groupBy("merchant_category") \
  .count() \
  .orderBy("count", ascending=False) \
  .show(10)

+-----------------+-----+
|merchant_category|count|
+-----------------+-----+
|       E-commerce|19932|
|        Groceries|15016|
|      Restaurants|10049|
|           Travel|10030|
|        Utilities|10017|
|             Fuel|10011|
|      Electronics| 9996|
|    Entertainment| 8085|
|       Healthcare| 6864|
+-----------------+-----+



In [0]:
from pyspark.sql.functions import sum as _sum, round

df.groupBy("channel") \
  .agg(
      round(_sum("amount"),2).alias("total_spend"),
      round((_sum("amount") / df.agg(_sum("amount")).collect()[0][0]) * 100, 2).alias("spend_share_pct")
  ) \
  .orderBy("total_spend", ascending=False) \
  .show()

+-------+--------------+---------------+
|channel|   total_spend|spend_share_pct|
+-------+--------------+---------------+
| Online|1.3021228802E8|          49.86|
|    POS| 9.173246217E7|          35.13|
|    ATM| 3.920258792E7|          15.01|
+-------+--------------+---------------+



In [0]:
df.groupBy("is_international") \
  .agg(round(_sum("amount"),2).alias("total_spend")) \
  .orderBy("is_international") \
  .show()

+----------------+--------------+
|is_international|   total_spend|
+----------------+--------------+
|               0|2.3469066334E8|
|               1| 2.645667477E7|
+----------------+--------------+



In [0]:
from pyspark.sql.functions import avg

df.groupBy("merchant_category") \
  .agg(round(avg("amount"),2).alias("avg_spend")) \
  .orderBy("avg_spend", ascending=False) \
  .show(10)

+-----------------+---------+
|merchant_category|avg_spend|
+-----------------+---------+
|        Utilities|  2653.16|
|           Travel|  2644.59|
|       Healthcare|  2633.79|
|      Restaurants|  2631.85|
|        Groceries|  2624.49|
|    Entertainment|  2601.27|
|      Electronics|  2590.19|
|             Fuel|  2579.33|
|       E-commerce|  2577.05|
+-----------------+---------+



In [0]:
from pyspark.sql.functions import to_timestamp, col


In [0]:
# Use to_timestamp with correct format (dd-MM-yyyy HH:mm)
df = df.withColumn(
    "txn_timestamp",
    to_timestamp(col("txn_timestamp"), "dd-MM-yyyy HH:mm")
)

In [0]:
from pyspark.sql.functions import expr

df = df.withColumn(
    "txn_timestamp",
    expr("try_to_timestamp(txn_timestamp, 'dd-MM-yyyy HH:mm')")
)

In [0]:
df.select("txn_timestamp").show(5, truncate=False)
df.printSchema()


+-------------------+
|txn_timestamp      |
+-------------------+
|2025-10-05 17:27:00|
|2025-06-09 02:13:00|
|2025-10-27 04:21:00|
|2025-06-29 14:13:00|
|2025-06-13 07:23:00|
+-------------------+
only showing top 5 rows
root
 |-- transaction_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- txn_timestamp: timestamp (nullable = true)
 |-- merchant_category: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- channel: string (nullable = true)
 |-- card_present: long (nullable = true)
 |-- is_international: long (nullable = true)
 |-- branch_id: string (nullable = true)



In [0]:
from pyspark.sql.functions import hour, dayofweek, avg, round, count, sum as _sum

# Hourly analysis
df.groupBy(hour("txn_timestamp").alias("txn_hour")) \
  .agg(
      round(avg("amount"), 2).alias("avg_amount"),
      count("*").alias("txn_count")
  ) \
  .orderBy("txn_hour") \
  .show()

# Day of week analysis
df.groupBy(dayofweek("txn_timestamp").alias("day_of_week")) \
  .agg(
      round(_sum("amount"), 2).alias("total_spend")
  ) \
  .orderBy("day_of_week") \
  .show()


+--------+----------+---------+
|txn_hour|avg_amount|txn_count|
+--------+----------+---------+
|       0|   2607.88|     4210|
|       1|   2582.96|     4085|
|       2|   2616.95|     4189|
|       3|   2614.68|     4132|
|       4|   2577.07|     4271|
|       5|   2668.14|     4121|
|       6|   2542.24|     4076|
|       7|   2576.41|     4155|
|       8|   2636.85|     4230|
|       9|   2620.96|     4191|
|      10|   2670.23|     4180|
|      11|   2617.66|     4241|
|      12|   2600.73|     4122|
|      13|   2644.76|     4168|
|      14|   2621.06|     4176|
|      15|   2576.74|     4132|
|      16|   2606.68|     4095|
|      17|   2614.47|     4149|
|      18|   2595.63|     4269|
|      19|   2620.75|     4250|
+--------+----------+---------+
only showing top 20 rows
+-----------+-------------+
|day_of_week|  total_spend|
+-----------+-------------+
|          1|3.758560255E7|
|          2|3.852824076E7|
|          3|3.643275741E7|
|          4|3.652067363E7|
|          

In [0]:
df.groupBy("branch_id") \
  .agg(
      round(_sum("amount"),2).alias("total_spend"),
      count("*").alias("txn_count")
  ) \
  .orderBy("total_spend", ascending=False) \
  .show(10)


+---------+-----------+---------+
|branch_id|total_spend|txn_count|
+---------+-----------+---------+
|     B035| 5614631.52|     2032|
|     B004| 5567550.87|     2014|
|     B044| 5523104.86|     2013|
|     B033| 5514028.32|     2065|
|     B012| 5496910.78|     2067|
|     B036| 5418054.73|     2057|
|     B029| 5383058.03|     2022|
|     B032| 5373375.49|     2019|
|     B030| 5348633.01|     2026|
|     B019| 5347595.87|     2036|
+---------+-----------+---------+
only showing top 10 rows


In [0]:
df.stat.corr("amount", "is_international")


0.006114433462035578

In [0]:
merchant_spend = (
    df.groupBy("merchant_category")
      .agg(round(sum("amount"), 2).alias("total_spend"))
      .orderBy(col("total_spend").desc())
)

display(merchant_spend)

merchant_category,total_spend
E-commerce,5.136578779E7
Groceries,3.940929779E7
Utilities,2.657674771E7
Travel,2.652520686E7
Restaurants,2.644750476E7
Electronics,2.589152211E7
Fuel,2.582170309E7
Entertainment,2.103122832E7
Healthcare,1.807833968E7


Databricks visualization. Run in Databricks to view.

In [0]:
channel_spend = (
    df.groupBy("channel")
      .agg(round(sum("amount"), 2).alias("total_spend"))
      .orderBy(col("total_spend").desc())
)

display(channel_spend)

channel,total_spend
Online,1.3021228802E8
POS,9.173246217E7
ATM,3.920258792E7


Databricks visualization. Run in Databricks to view.

In [0]:
hourly_txns = (
    df.groupBy(hour("txn_timestamp").alias("txn_hour"))
      .agg(count("*").alias("txn_count"))
      .orderBy("txn_hour")
)

display(hourly_txns)

txn_hour,txn_count
0,4210
1,4085
2,4189
3,4132
4,4271
5,4121
6,4076
7,4155
8,4230
9,4191


Databricks visualization. Run in Databricks to view.